In [1]:
!pip install session_info
!pip install pycaret

  Using cached session_info-1.0.0-py3-none-any.whl
  Using cached stdlib_list-0.9.0-py3-none-any.whl (75 kB)
  Using cached pycaret-3.0.3-py3-none-any.whl (484 kB)
  Using cached plotly_resampler-0.8.3.2-cp39-cp39-manylinux_2_31_x86_64.whl
  Using cached xxhash-3.2.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached plotly-5.15.0-py2.py3-none-any.whl (15.5 MB)
  Using cached schemdraw-0.15-py3-none-any.whl (106 kB)
  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)
  Using cached tbats-1.1.3-py3-none-any.whl (44 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached lightgbm-3.3.5-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached wurlitzer-3.0.3-py3-none-any.whl (7.3 kB)
  Using cached category_encoders-2.6.1-py2.py3-none-any.whl (81 kB)
  Using cached sktime-0.20.0-py3-none-any.whl (17.0 MB)
  Using cached pyod-1.1.0-py3-none-any.whl
  Using cached pmdarima-2.0.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x

In [2]:
pip install pycaret[tuners]

  Using cached tune_sklearn-0.4.5-py3-none-any.whl (41 kB)
  Using cached scikit_optimize-0.9.0-py2.py3-none-any.whl (100 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached ray-2.5.1-cp39-cp39-manylinux2014_x86_64.whl (56.2 MB)
  Using cached optuna-3.2.0-py3-none-any.whl (390 kB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached future-0.18.3-py3-none-any.whl
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached cmaes-0.9.1-py3-none-any.whl (21 kB)
  Using cached frozenlist-1.3.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (158 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached pyarrow-12.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39.0 MB)
  Using cached tensorboardX-2.6.1-py2.py3-none-any.whl (101 kB)
  Using cached pyaml-23.5.9-py3-none-any.whl (17 kB)
  Using ca

#### Import librairies

In [3]:
import pandas as pd
import os
import re


from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset , random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

import random

from pycaret.classification import *
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

import session_info

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
session_info.show()

#### Set the random seed for reproducibility

In [4]:
seed = 64
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#### Get the data

In [5]:
raw_data= pd.read_csv('data/train.csv', low_memory=False)
data_test= pd.read_csv('data/test.csv', low_memory=False)

In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 365 entries, Patient_ID to Type_of_Venom_Allergy_IGE_Venom
dtypes: float64(322), int64(32), object(11)
memory usage: 8.3+ MB


#### Looking which are the targets to predict

In [7]:
missing_cols = set(raw_data.columns) ^ set(data_test.columns)
print(missing_cols)
len(missing_cols)

{'Severe_Allergy', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Other', 'Type_of_Food_Allergy_Shellfish', 'Type_of_Food_Allergy_Fish', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_GINA', 'Type_of_Food_Allergy_Mammalian_Milk', 'Food_Allergy', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Tree_Nuts', 'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram', 'trustii_id', 'Type_of_Food_Allergy_Other_Legumes', 'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_ARIA', 'Allergy_Present', 'Type_of_Food_Allergy_Egg', 'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Aromatics', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Peanut', 'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Food_

30

## Data Pre-processing

### Preprocessing for the train set

In [8]:
liste_of_Targets =['Allergy_Present', 'Severe_Allergy', 'Respiratory_Allergy', 'Food_Allergy', 'Venom_Allergy',
                     'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_CONJ', 
                     'Type_of_Respiratory_Allergy_GINA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                     'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree',
                     'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach',
                     'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Aromatics', 'Type_of_Food_Allergy_Other',
                     'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish',
                     'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Food_Allergy_Mammalian_Milk', 
                     'Type_of_Food_Allergy_Oral_Syndrom', 'Type_of_Food_Allergy_Other_Legumes', 'Type_of_Food_Allergy_Peanut',
                     'Type_of_Food_Allergy_Shellfish', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Tree_Nuts',
                     'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Venom_Allergy_IGE_Venom']
def preprocessing_data(df):
    df = df.drop('Food_Type_0', axis =1)
    df.replace(-1, 0, inplace=True)
    data_noNAN = df.fillna(-1)
    # obtain Targets
    Targets = data_noNAN.loc[:,liste_of_Targets]
    # filter feautures
    X1=data_noNAN.loc[:, ['Chip_Type','Age','Gender','French_Residence_Department','Blood_Month_sample']]
    X= data_noNAN.iloc[:, 8:-29]
    data = pd.concat( [X1, X] , axis=1)
    # handle the 'Treatment_of_rhinitis' feature
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].astype(str)
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].str.replace('.0', '', regex=True)
    
    ##  Get_dummies of the 'object' type columns
    
    columns_to_encode = ['Chip_Type', 'French_Residence_Department', 'French_Region',
         'Treatment_of_athsma', 'Age_of_onsets',
       'General_cofactors', 'Treatment_of_atopic_dematitis','Treatment_of_rhinitis']
    
    ### Split the columns using multiple delimiters and create dummy columns
    dummy_dfs = []
    for col in columns_to_encode:
        # Split the data in the column that use  delimiters
        data[col] = data[col].astype(str)
        data[col] = data[col].apply(lambda x: [i.strip() for i in re.split('[,.]', x)])

        # Create dummy columns
        dummy_df = pd.get_dummies(data[col].apply(pd.Series).stack(), prefix=f"{col}", prefix_sep='_').groupby(level=0).sum()
        dummy_dfs.append(dummy_df)

    ### Concatenate the original DataFrame with the dummy columns
    df_final = pd.concat([data] + dummy_dfs, axis=1)

    ### Drop the original columns from the final dataset
    df_final.drop(columns=columns_to_encode, inplace=True)
    
    # Converting all values into 'float16' type
    encode_data = df_final.astype('float16')
    print(encode_data.info())
    
    return encode_data,Targets

In [9]:
encode_data,Targets = preprocessing_data(raw_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 467 entries, Age to Treatment_of_rhinitis_9
dtypes: float16(467)
memory usage: 2.7 MB
None


### Preprocessing for the test set

In [10]:
def preprocessing_data_test(df):
    df = df.drop('Food_Type_0', axis =1)
    df.replace(-1, 0, inplace=True)
    data_test_noNAN = df.fillna(-1)
    # filter feautures
    X1=data_test_noNAN.loc[:, ['Chip_Type']]
    X= data_test_noNAN.iloc[:, 5:]
    data = pd.concat( [X1, X] , axis=1)
    # handle the 'Treatment_of_rhinitis' feature
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].astype(str)
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].str.replace('.0', '', regex=True)
    # handle 'Age_of_onsets' which don't have the same format in data test and train
    data['Age_of_onsets'] = data['Age_of_onsets'].astype(str)

    
    ##  Get_dummies of the 'object' type columns
    
    columns_to_encode = ['Chip_Type', 'French_Residence_Department', 'French_Region',
         'Treatment_of_athsma', 'Age_of_onsets',
       'General_cofactors', 'Treatment_of_atopic_dematitis','Treatment_of_rhinitis']
    
    ### Split the columns using multiple delimiters and create dummy columns
    dummy_dfs = []
    for col in columns_to_encode:
        # Split the data in the column that use  delimiters
        data[col] = data[col].astype(str)
        data[col] = data[col].apply(lambda x: [i.strip() for i in re.split('[,.]', x)])

        # Create dummy columns
        dummy_df = pd.get_dummies(data[col].apply(pd.Series).stack(), prefix=f"{col}", prefix_sep='_').groupby(level=0).sum()
        dummy_dfs.append(dummy_df)

    ### Concatenate the original DataFrame with the dummy columns
    df_final = pd.concat([data] + dummy_dfs, axis=1)

    ### Drop the original columns from the final dataset
    df_final.drop(columns=columns_to_encode, inplace=True)
    
    # Converting all values into 'float16' type
    encode_data = df_final.astype('float16')
    print(encode_data.info())
    
    return encode_data


In [11]:
encode_data_test = preprocessing_data_test(data_test)
missing_cols = set(encode_data.columns) ^ set(encode_data_test.columns)
print(missing_cols)
len(missing_cols)
encode_data_test = encode_data_test.reindex(columns=encode_data.columns, fill_value=0).astype('float16')
encode_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Columns: 444 entries, Age to Treatment_of_rhinitis_9
dtypes: float16(444)
memory usage: 508.3 KB
None
{'Treatment_of_atopic_dematitis_7', 'French_Residence_Department_deptNNN', 'French_Residence_Department_deptCCCC', 'French_Residence_Department_deptP', 'French_Region_regionN', 'Treatment_of_athsma_10', 'French_Residence_Department_deptHHH', 'French_Residence_Department_deptW', 'French_Residence_Department_deptRRR', 'General_cofactors_11', 'French_Region_regionO', 'French_Residence_Department_deptPPP', 'French_Residence_Department_deptAAAA', 'French_Residence_Department_deptU', 'French_Residence_Department_deptK', 'French_Residence_Department_deptTTT', 'French_Residence_Department_deptZZZ', 'French_Residence_Department_deptUU', 'French_Residence_Department_deptDD', 'French_Residence_Department_deptIII', 'French_Residence_Department_deptOOO', 'French_Residence_Department_deptDDD', 'Treatment_of_athsma_8', 'French_Re

#### Pycaret part

In [18]:
def obtain_stack_methods_imbl(encode_data, Y):
    
    liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']
        
    folder_path = f'Test_Imbl_stacked_multi_tuned_selection'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for column in Y.columns:
        if column not in liste_column_monovalue :
            rows_with_9 = Y[column].isin([9])
            Targets_without_9 = Y[column][~rows_with_9].reset_index(drop=True)
            X = encode_data[~rows_with_9].reset_index(drop=True)

            dataset= pd.concat([X,Targets_without_9], axis = 1)

            setup_model = setup(data= dataset, target= column, train_size=0.82, fix_imbalance=True, fix_imbalance_method='SMOTE')
            lightgbm= create_model('lightgbm')
            tuned_lightgbm, tuner= tune_model(lightgbm, n_iter = 70, optimize = 'F1', fold = 8, search_library = 'tune-sklearn', search_algorithm = 'hyperopt', return_tuner=True)
            
            file_name = f"{column}_imblearn_stacked_multi_tuned_selection"
            file_path = os.path.join(folder_path, file_name)
            save_model(tuned_lightgbm, file_path)
        else:
            continue

In [19]:
obtain_stack_methods_imbl(encode_data, Targets)

(_Trainable pid=66047) [LightGBM] [Warning] bagging_fraction is set=0.49247705785158014, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49247705785158014 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] feature_fraction is set=0.532383789354755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.532383789354755 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] bagging_fraction is set=0.49247705785158014, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49247705785158014 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] feature_fraction is set=0.532383789354755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.532383789354755 [repeated 2x across cluster]
(_Trainable pid=66047) [LightG

2023-06-26 20:44:39,911	INFO tune.py:1111 -- Total run time: 1440.25 seconds (1440.19 seconds for the tuning loop).


Transformation Pipeline and Model Successfully Saved


In [14]:
def obtain_blended_models(encode_data, Y):
    liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']
        
    folder_path = f'Test_Imbl_blended_3_model_1'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for column in Y.columns:
        if column not in liste_column_monovalue :
            rows_with_9 = Y[column].isin([9])
            Targets_without_9 = Y[column][~rows_with_9].reset_index(drop=True)
            X = encode_data[~rows_with_9].reset_index(drop=True)

            dataset= pd.concat([X,Targets_without_9], axis = 1)
            setup_model = setup(data= dataset, target= column, train_size=0.70, fix_imbalance=True, fix_imbalance_method='SMOTETomek')
            
            # Recall the models from different folders
            model_folder_tune_sklearn = f'Test_Imbl_stacked_multi_tuned_selection/{column}_imblearn_stacked_multi_tuned_selection'
            model_folder_smote_default = f'Test_Imbl_SMOTE_tuned/{column}_ETC_brutforce_imb_binary'
            model_folder_BorderlineSMOTE_default = f'Test_Imbl_BorderlineSMOTE_tuned/{column}_ETC_brutforce_imb_binary'

            model_1 = load_model(model_folder_tune_sklearn)
            model_2 = load_model(model_folder_smote_default)
            model_3 = load_model(model_folder_BorderlineSMOTE_default)

            # Blend the recalled models
            blended_model = blend_models(estimator_list=[model_1, model_2,model_3])
            
            file_name = f"{column}_imblearn_blended_3_models"
            file_path = os.path.join(folder_path, file_name)
            save_model(blended_model, file_path)

In [15]:
obtain_blended_models(encode_data, Targets)

,Description,Value
0,Session id,7951
1,Target,Allergy_Present
2,Target type,Binary
3,Original data shape,"(1759, 468)"
4,Transformed data shape,"(2430, 468)"
5,Transformed train set shape,"(1902, 468)"
6,Transformed test set shape,"(528, 468)"
7,Numeric features,467
8,Preprocess,1
9,Imputation type,simple


Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8306,0.9107,0.8958,0.8866,0.8912,0.5094,0.5096
1,0.8049,0.8731,0.8646,0.8830,0.8737,0.4453,0.4458
2,0.7805,0.8561,0.8542,0.8632,0.8586,0.3678,0.3679
3,0.8130,0.9163,0.8750,0.8842,0.8796,0.4615,0.4616
4,0.9268,0.9545,0.9583,0.9485,0.9534,0.7836,0.7838
5,0.8780,0.9078,0.9688,0.8857,0.9254,0.5957,0.6140
6,0.8293,0.8387,0.8958,0.8866,0.8912,0.4950,0.4952
7,0.8699,0.9218,0.9474,0.8911,0.9184,0.5998,0.6067
8,0.8130,0.8477,0.9158,0.8529,0.8832,0.4168,0.4235


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


...

,Description,Value
0,Session id,634
1,Target,Type_of_Venom_Allergy_IGE_Venom
2,Target type,Binary
3,Original data shape,"(2989, 468)"
4,Transformed data shape,"(5051, 468)"
5,Transformed train set shape,"(4154, 468)"
6,Transformed test set shape,"(897, 468)"
7,Numeric features,467
8,Preprocess,1
9,Imputation type,simple


Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9905,0.8317,0.5000,0.5000,0.5000,0.4952,0.4952
1,0.9810,0.9495,0.5000,0.2500,0.3333,0.3248,0.3450
2,0.9809,0.2404,0.0000,0.0000,0.0000,-0.0072,-0.0084
3,0.9665,0.9375,0.0000,0.0000,0.0000,-0.0083,-0.0119
4,0.9856,0.9856,0.0000,0.0000,0.0000,-0.0064,-0.0068
5,0.9904,0.9904,1.0000,0.3333,0.5000,0.4964,0.5746
6,0.9761,1.0000,1.0000,0.1667,0.2857,0.2798,0.4033
7,0.9713,0.9058,0.0000,0.0000,0.0000,-0.0129,-0.0137
8,0.9856,0.9831,0.5000,0.3333,0.4000,0.3930,0.4013


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


### Generate predictions from classifiers and include them into the dataset + evaluation 

In [16]:
data= pd.read_csv('data/train.csv', low_memory=False)
data_test= pd.read_csv('data/test.csv', low_memory=False)

In [23]:
List_methods_imbl=['SMOTE']

def obtain_pred(data, folder_path):
    liste=[]
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            print("Calling model:", file) 
            model = load_model(os.path.join(subdir, os.path.splitext(file)[0]))
            values=predict_model(model, data= data.reset_index())
            file_name = os.path.splitext(file)[0]
            values_name = '{}_{}'.format(file_name, 'values')
            globals()[values_name] = values
            globals()[values_name] = globals()[values_name].rename(columns={'prediction_label': 'pred_label ' + str(values_name), 'prediction_score': 'pred_score ' + str(values_name) })
            liste.append(globals()[values_name])
        return liste
    
def obtain_all_pred_methods(data, list_of_methods):
    all_lists = {} 
    for method in list_of_methods:
        folder_path = f'Test_Imbl_{method}_tuned_stacked_and_calibrated'
        all_lists[f"{method}_tuned"] = obtain_pred(data, folder_path)
    return all_lists
    

In [17]:
def obtain_pred(data, folder_path):
    liste=[]
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            print("Calling model:", file) 
            model = load_model(os.path.join(subdir, os.path.splitext(file)[0]))
            values=predict_model(model, data= data.reset_index())
            file_name = os.path.splitext(file)[0]
            values_name = '{}_{}'.format(file_name, 'values')
            globals()[values_name] = values
            globals()[values_name] = globals()[values_name].rename(columns={'prediction_label': 'pred_label ' + str(values_name), 'prediction_score': 'pred_score ' + str(values_name) })
            liste.append(globals()[values_name])
        return liste

In [18]:
dico_df= obtain_pred(encode_data, folder_path=f'Test_Imbl_blended_3_model_1')

Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Gram_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Venom_Allergy_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Herb_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Venom_Allergy_ATCD_Venom_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Venom_Allergy_IGE_Venom_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


...

Calling model: Allergy_Present_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_GINA_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Tree_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_Other_Legumes_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


In [19]:
print(len(dico_df))
dico_df[0]

#len(dico_df.values())

27


,index,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,Act_d_5,...,Treatment_of_atopic_dematitis_7,Treatment_of_atopic_dematitis_9,Treatment_of_rhinitis_0,Treatment_of_rhinitis_1,Treatment_of_rhinitis_2,Treatment_of_rhinitis_3,Treatment_of_rhinitis_4,Treatment_of_rhinitis_9,pred_label Type_of_Respiratory_Allergy_IGE_Pollen_Gram_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_IGE_Pollen_Gram_imblearn_blended_3_models_values
0,0,15.0,0.0,7.0,1.0,1.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.6657
1,1,72.0,1.0,5.0,9.0,1.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.6734
2,2,67.0,1.0,6.0,9.0,0.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.6737
3,3,13.0,1.0,9.0,1.0,1.0,9.0,0.000000,2.150391,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0.6623
4,4,28.0,1.0,12.0,9.0,1.0,1.0,0.000000,74.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.6622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,2984,21.0,1.0,3.0,9.0,1.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.6464
2985,2985,3.0,0.0,1.0,9.0,0.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.6742
2986,2986,15.0,0.0,12.0,1.0,1.0,1.0,0.000000,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.6685
2987,2987,11.0,1.0,4.0,0.0,1.0,0.0,0.770020,0.099976,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.6731


In [20]:

liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df = pd.DataFrame()
for i,df in enumerate(dico_df):
    if i==0:
        merged_df = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df = pd.concat([merged_df, last_two_columns], axis=1)

added_list = ['pred_label ' + s + '_imblearn_blended_3_models_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df)), columns= added_list)
merged_df = pd.concat([merged_df, zeros_data], axis=1)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 524 entries, index to pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_blended_3_models_values
dtypes: float32(467), float64(27), int32(1), int64(29)
memory usage: 6.6 MB


In [21]:
data_true= pd.read_csv('data/train.csv', low_memory=False)

In [22]:
merged_df.head()

,index,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,Act_d_5,...,pred_label Type_of_Respiratory_Allergy_GINA_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_GINA_imblearn_blended_3_models_values,pred_label Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_imblearn_blended_3_models_values,pred_label Type_of_Respiratory_Allergy_IGE_Pollen_Tree_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_IGE_Pollen_Tree_imblearn_blended_3_models_values,pred_label Type_of_Food_Allergy_Other_Legumes_imblearn_blended_3_models_values,pred_score Type_of_Food_Allergy_Other_Legumes_imblearn_blended_3_models_values,pred_label Type_of_Food_Allergy_Other_imblearn_blended_3_models_values,pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_blended_3_models_values
0,0,15.0,0.0,7.0,1.0,1.0,0.0,0.0,0.000000,0.0,...,0,0.8170,0,0.9924,0,0.9978,0,0.6603,0,0
1,1,72.0,1.0,5.0,9.0,1.0,9.0,0.0,0.000000,0.0,...,1,0.5716,0,0.9918,0,0.6787,0,0.6702,0,0
2,2,67.0,1.0,6.0,9.0,0.0,9.0,0.0,0.000000,0.0,...,1,0.5921,0,0.9985,0,0.9988,0,0.6679,0,0
3,3,13.0,1.0,9.0,1.0,1.0,9.0,0.0,2.150391,0.0,...,1,0.8104,1,0.8915,1,0.9970,0,0.6366,0,0
4,4,28.0,1.0,12.0,9.0,1.0,1.0,0.0,74.000000,0.0,...,1,0.6426,1,0.8417,1,0.9906,0,0.6431,0,0


#### Evaluation binary for each methods

In [23]:
import numpy as np
target_columns = ['Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Other_Legumes', 'Allergy_Present', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_Other',
                  'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Food_Allergy', 'Type_of_Food_Allergy_Oral_Syndrom','Type_of_Food_Allergy_Tree_Nuts', 'Severe_Allergy',
                  'Type_of_Food_Allergy_Aromatics', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut',
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_GINA', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast',
                  'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Food_Allergy_Shellfish']


f1_scores=[]
below_threshold_columns=[]
threshold =0.85
for column in target_columns:
    rows_with_9 = data_true[column].isin([9])
    Targets_without_9 = data_true[column][~rows_with_9]
    merged_df_removal = merged_df[~rows_with_9]

    merged_df_mod = pd.concat([merged_df_removal, Targets_without_9] , axis=1).reset_index(drop=True)
    y_true = merged_df_mod[column]
    y_pred = merged_df_mod['pred_label '+ column + '_imblearn_blended_3_models_values']

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate recall
    #recall = recall_score(y_true, y_pred)

    # Calculate F1 score
    if (data_true[column] == 1).any():
        f1 = f1_score(y_true, y_pred,average='macro')
    else:
        f1 = f1_score(y_true, y_pred, pos_label=0)
    cm = confusion_matrix(y_true, y_pred)

    # Display the metrics and confusion matrix
    print(f"Metrics for {column}:")
    print(f"Accuracy: {accuracy}")
    #print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print("Confusion Matrix:")
    print(cm)
    print()
    if f1 != 0:
        f1_scores.append(f1) # Append the F1 score to the list
    if f1 < threshold:
        below_threshold_columns.append(column)

mean_f1 = sum(f1_scores) / len(f1_scores)  # Calculate the mean of F1 scores

print(f"Mean F1 Score: {mean_f1}")
print(below_threshold_columns)
print('')

Metrics for Type_of_Food_Allergy_Cereals_&_Seeds:
Accuracy: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1607]]

Metrics for Type_of_Food_Allergy_Other_Legumes:
Accuracy: 0.9695084007467331
F1 Score: 0.7713836943614578
Confusion Matrix:
[[1527   30]
 [  19   31]]

Metrics for Allergy_Present:
Accuracy: 0.9499715747583855
F1 Score: 0.9264316158885505
Confusion Matrix:
[[ 338   53]
 [  35 1333]]

Metrics for Type_of_Food_Allergy_Mammalian_Milk:
Accuracy: 0.9900435594275047
F1 Score: 0.8070017414279709
Confusion Matrix:
[[1578    4]
 [  12   13]]

Metrics for Type_of_Food_Allergy_Other:
Accuracy: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1607]]

Metrics for Type_of_Respiratory_Allergy_IGE_Mite_Cockroach:
Accuracy: 0.9754479097544791
F1 Score: 0.9752088015668177
Confusion Matrix:
[[809  18]
 [ 19 661]]

Metrics for Venom_Allergy:
Accuracy: 0.9809300769488123
F1 Score: 0.695155532891382
Confusion Matrix:
[[2913   43]
 [  14   19]]

Metrics for Type_of_Respiratory_Allergy_ARIA:
Accuracy: 0.948241539482

## Generate prediction for the test set

In [24]:
original_validation_data = pd.read_csv('data/test.csv')

data_test = original_validation_data.set_index('trustii_id')

In [25]:
encode_data_test = preprocessing_data_test(data_test)
missing_cols = set(encode_data.columns) ^ set(encode_data_test.columns)
print(missing_cols)
len(missing_cols)
encode_data_test = encode_data_test.reindex(columns=encode_data.columns, fill_value=0).astype('float16')
encode_data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586 entries, 1 to 1282
Columns: 443 entries, Gender to Treatment_of_rhinitis_9
dtypes: float16(443)
memory usage: 527.8 KB
None
{'Treatment_of_atopic_dematitis_7', 'French_Residence_Department_deptNNN', 'French_Residence_Department_deptCCCC', 'French_Residence_Department_deptP', 'French_Region_regionN', 'Treatment_of_athsma_10', 'French_Residence_Department_deptHHH', 'French_Residence_Department_deptW', 'French_Residence_Department_deptRRR', 'General_cofactors_11', 'French_Region_regionO', 'French_Residence_Department_deptPPP', 'French_Residence_Department_deptAAAA', 'Age', 'French_Residence_Department_deptU', 'French_Residence_Department_deptK', 'French_Residence_Department_deptTTT', 'French_Residence_Department_deptZZZ', 'French_Residence_Department_deptUU', 'French_Residence_Department_deptDD', 'French_Residence_Department_deptIII', 'French_Residence_Department_deptOOO', 'French_Residence_Department_deptDDD', 'Treatment_of_athsma_8',

In [26]:
dico_df= obtain_pred(encode_data_test.reset_index(), folder_path=f'Test_Imbl_blended_3_model_1')

Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Gram_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Venom_Allergy_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Herb_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Venom_Allergy_ATCD_Venom_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Venom_Allergy_IGE_Venom_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


...

Calling model: Allergy_Present_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_GINA_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Pollen_Tree_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_Other_Legumes_imblearn_blended_3_models.pkl
Transformation Pipeline and Model Successfully Loaded


In [27]:
print(len(dico_df))


27


In [28]:
liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df = pd.DataFrame()
for i,df in enumerate(dico_df):
    if i==0:
        merged_df = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df = pd.concat([merged_df, last_two_columns], axis=1)

added_list = ['pred_label ' + s + '_imblearn_blended_3_models_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df)), columns= added_list)
merged_df = pd.concat([merged_df, zeros_data], axis=1)
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Columns: 525 entries, index to pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_blended_3_models_values
dtypes: float32(467), float64(27), int32(2), int64(29)
memory usage: 1.3 MB


In [29]:
merged_df.head()

,index,trustii_id,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,...,pred_label Type_of_Respiratory_Allergy_GINA_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_GINA_imblearn_blended_3_models_values,pred_label Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_IGE_Mite_Cockroach_imblearn_blended_3_models_values,pred_label Type_of_Respiratory_Allergy_IGE_Pollen_Tree_imblearn_blended_3_models_values,pred_score Type_of_Respiratory_Allergy_IGE_Pollen_Tree_imblearn_blended_3_models_values,pred_label Type_of_Food_Allergy_Other_Legumes_imblearn_blended_3_models_values,pred_score Type_of_Food_Allergy_Other_Legumes_imblearn_blended_3_models_values,pred_label Type_of_Food_Allergy_Other_imblearn_blended_3_models_values,pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_blended_3_models_values
0,0,1,0.0,1.0,6.0,9.0,1.0,0.0,0.0,0.000000,...,1,0.8087,1,0.9974,1,0.9959,0,0.6432,0,0
1,1,4,0.0,1.0,7.0,9.0,1.0,1.0,0.0,0.109985,...,1,0.5226,1,0.9865,0,0.9937,0,0.6242,0,0
2,2,5,0.0,0.0,10.0,1.0,1.0,1.0,0.0,0.000000,...,0,0.5009,0,0.9705,1,0.9807,0,0.6509,0,0
3,3,7,0.0,1.0,8.0,0.0,1.0,0.0,0.0,0.000000,...,0,0.7784,0,0.9808,0,0.9993,0,0.6475,0,0
4,4,8,0.0,0.0,4.0,1.0,0.0,9.0,0.0,0.000000,...,0,0.7957,0,0.9988,0,0.9995,0,0.6282,0,0


In [30]:
target_columns = ['Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Other_Legumes', 'Allergy_Present', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_Other',
                  'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Food_Allergy', 'Type_of_Food_Allergy_Oral_Syndrom','Type_of_Food_Allergy_Tree_Nuts', 'Severe_Allergy',
                  'Type_of_Food_Allergy_Aromatics', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut',
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_GINA', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast',
                  'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Food_Allergy_Shellfish']

original_validation_data = pd.read_csv('data/test.csv')
for elem in target_columns:
    original_validation_data[elem]=  merged_df['pred_label '+ elem + '_imblearn_blended_3_models_values']

In [31]:
len(original_validation_data)

586

In [32]:
original_validation_data.head(30)

,trustii_id,Patient_ID,Chip_Code,Chip_Type,Chip_Image_Name,Age,Gender,Blood_Month_sample,French_Residence_Department,French_Region,...,Type_of_Food_Allergy_Egg,Type_of_Food_Allergy_Fish,Type_of_Respiratory_Allergy_GINA,Respiratory_Allergy,Type_of_Food_Allergy_TPO,Type_of_Respiratory_Allergy_IGE_Pollen_Tree,Type_of_Food_Allergy_Fruits_and_Vegetables,Type_of_Respiratory_Allergy_IGE_Molds_Yeast,Type_of_Respiratory_Allergy_IGE_Dander_Animals,Type_of_Food_Allergy_Shellfish
0,1,PMP0156,22 262C 3858,ISAC_V2,NaN,8.0,1.0,6.0,deptBBB,regionJ,...,0,0,1,1,0,1,0,0,1,0
1,4,PCR0234,02AHX0DC,ALEX,02AHX0DC.bmp,14.0,1.0,7.0,deptL,regionD,...,1,0,1,1,0,0,0,0,1,0
2,5,PCR0532,02AUN372,ALEX,02AUN372.png,32.0,0.0,10.0,deptUUU,regionF,...,0,0,0,1,0,1,0,0,1,0
3,7,GJH0147,EKF3830_4,ISAC_V2,EKF3830_4_2200444337_2023_2_17_11_58_24.bmp,65.0,1.0,8.0,deptQ,regionF,...,0,0,0,0,0,0,0,1,0,0
4,8,TXV0009,881204001164,ISAC_V1,1G20027_2_881204001164_2012_4_25_18_32_58.bmp,5.0,0.0,4.0,deptII,regionC,...,0,0,0,0,0,0,0,0,0,0
5,9,PCR0118,02AFA752,ALEX,NaN,49.0,0.0,1.0,deptXXX,regionI,...,0,0,0,1,0,0,0,0,0,0
6,10,QVW0214,AB02627_3,ISAC_V1,NaN,6.0,1.0,2.0,deptY,regionD,...,0,0,1,1,0,1,0,1,1,0
7,15,TXV0157,881602013302,ISAC_V1,BAF4027_4_881602013302_2016_2_23_16_38_11.bmp,13.0,1.0,2.0,deptRR,regionB,...,0,0,1,1,0,0,0,0,0,0
8,18,WQW0190,223112546,ISAC_V2,END0E30_1_223112546_2023_1_3_16_20_19.bmp,12.0,0.0,11.0,deptOO,regionL,...,0,0,0,0,0,0,0,0,0,0
9,23,TXV0282,881903001372,ISAC_V1,CXG1527_3_881903001372_2019_3_14_3_51_59.bmp,8.0,0.0,3.0,deptEE,regionC,...,0,0,1,1,0,0,0,0,0,0


In [34]:
original_validation_data.to_csv(f"Submission_ML_pycaret_blended_3_models_binary.csv", index=False, encoding='UTF-8')

#### Logics is the same than the first submission but an fix_imbalanced_method is set to give models more samples from the minority class and less from the majority class.
#### one notebook for the the best models that we have obtain and blended together.

##### methods used to make the models: ipynb_file_path='Test_Pycaret_best_tuned_methods_selections_submission_binary.ipynb'

##### ipynb_file_path= 'Test_Pycaret_best_3_sampling_methods_submission_bina.ipynb'